<p style="font-weight:bold; font-size: 40px; color:#0171ff; padding-top: 50px; text-align:center"> How to create a new Scenario </p>

The International Financial Reporting Standard (IFRS) 17 dictates companies to  

> "... (a) exposures to risks and how they arise;

> (b) objectives, policies and processes for managing risks and the methods used to measure those risks;

>(c) concentrations of risk arising from insurance contracts;

>(d) the claims development—ie actual claims compared with previous estimates of the undiscounted amount of the claims;

>(e) the credit quality of reinsurance contract assets; and

>(f) liquidity risk, including a maturity analysis showing the estimated cash flows arising from insurance contracts... [1]"

but it does not force or prescribe *how* each Scenario should be modeled and *how many* or them should be considered.

Systemorph IFRS17 Solution gives the User the possibility to interpretate the requirements with the highest flexibility possible thanks to its open source approach.

In this notebook we focus on the creation of new Scenario and new AoC step to host them. Given some simple rules, the solution automatically incorporate new arbitrary AoC steps and then calculate PVs, CSMs and FP, calculating the Deltas from cumulative cash flows as usual.

# Initialization


Before performing any change, we reuse the standard Initialization code to create the Database.

In [0]:
#!eval-notebook "Initialization"

Now we want to import new Scenarios to study the effect of an increase of inflation of 10 percent points, using three different approaches: 
* Non-Financial Inflation Up 10pct: the cash flow is increased of 10 percent point for Premiums, Claims and Risk Adjustment;
* Financial Inflation Up 10pct: a new Aoc step is created to accomodated the specific effect of inflation;
* Yield Curve Up 10pct: a new Yield Curve is imported, where the expected incrementation of inflation is included.

The new Dimensions can be found in the file *Dimensions_EP3.xlsx*. Since we are adding new element to the dimensions *AoCType* and *Scenario*, we don't need to reimport the all Database but simply updating the relevant types.

In [0]:
await Import.FromFile("Dimensions_EP3.xlsx").WithType<AocType>().WithType<Scenario>().WithTarget(DataSource).ExecuteAsync() 

The AoC Configuration contains default values for the new AoC step. We reimport it again to let the Calculation Engine rebuild the total AoC chain.

In [0]:
await Import.FromFile("Dimensions_EP3.xlsx").WithFormat(ImportFormats.AocConfiguration).WithTarget(DataSource).ExecuteAsync()

# Import of cash flows

First, we import the BE TD. Since we are studying a BBA Datanode, the locked economic basis is used. It means that discounting of the Present Values (PV) is done with the Yield Curve related with the Period (year and month) when the Datanode was activated (imported for the first time). This allow us to import the cash flow without importing a zield curve for the same Period.

In [0]:
await Import.FromFile("A_G_2023_3.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("CF_G_2023_3.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

Financial and Non-Financial Scenarios calculations are triggered by importing the respective cash flows. The CE has been designed to avoid mixing sources of perturbation. Therefore, each import of Scenario TD will use always the BE yield curve. Viceversa, each Scenario yield curve will discount only BE TD. 

In [0]:
await Import.FromFile("CF_G_2023_3_NFIIUUP.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("CF_G_2023_3_FIIUUP.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var testYC = @"
@@Main,
Year,Month,Scenario
2023,3,YCIIUP
@@YieldCurve
Currency,Values0
CHF,1.068685495
EUR,1.068685495";

In [0]:
//await Import.FromFile("YieldCurve_YCUP.xlsx").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()
await Import.FromString(testYC).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
DataSource.Query<YieldCurve>().ToArray()

In [0]:
DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArray()

In [0]:
Workspace.Reset(x => x.ResetInitializationRules());
Workspace.InitializeFrom(DataSource);

In [0]:
var ifrs17 = new Ifrs17(Workspace, Scopes, Report);

# Reimport

Real business life is different from linear, educational exercesises. Usually data does not come together, or it is found to be wrong; reimporting new inputs to upload previous data is the standard more than the exception. Because of interdependences between BE and Scenario, and between cash flow and yield curve, changing some data in a Partition may outdate something other. The CE takes care of these problems, retriggering calculation when needed.


<span style="color:red"> CRITICAL:</span> Because of the periodic nature of IFRS17 reporting process, the import of any input file (cash flow, actuals, yield curve) does trigger calculations *only in the same Period*. 
This implies that data from following Periods, if present, could potentially be outdated.
*Do we want to suggest a best practise in case of reopening of previous years?*


In [0]:
await Import.FromFile("CF_G_2023_3.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

A warning informs the user which Scenario will be recalculated.

# Reports

In [0]:
var pv = ifrs17.PresentValues;
pv.DataFilter = new [] {("GroupOfContract", "GIC2")};
pv.Scenario = "All";
await pv.ToReportAsync

In [0]:
pv.Scenario = "Delta";
await pv.ToReportAsync

In [0]:
var csm = ifrs17.AllocatedTechnicalMargins;
csm.ReportingPeriod = (2023, 3);
csm.DataFilter = new [] {("GroupOfContract", "GIC2")};
csm.Scenario = "All";
await csm.ToReportAsync

In [0]:
var fp = ifrs17.FinancialPerformance;
fp.Scenario = "All";
await fp.ToReportAsync